In [4]:
import math
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scienceplots

In [5]:
RUNS = os.listdir("runs/")

TAGS = [
    "train/loss",
    "train/accuracy",
    "Accuracy_train",
    "validation/loss",
    "validation/accuracy"
]

LABELS = [
    "Training Loss",
    "Training Accuracy",
    "Validation Loss",
    "Validation Accuracy"
]

In [ ]:
# WIP

with plt.style.context('science'):
    plt.figure(figsize=(12, 12), dpi=300)

    for i, tag in enumerate(TAGS):
        resnet_ds = pd.read_csv(f"runs/run-ResNet_2024-05-26_18-55-35-tag-{tag}.csv")
        plainnet_ds = pd.read_csv(f"runs/run-PlainNet_2024-05-26_19-20-18-tag-{tag}.csv")

        steps = np.array(resnet_ds["Step"])
        n = 10 if tag[-5:] == "train" else 5

        resnet_values = np.array(resnet_ds["Value"])
        resnet_values_ext = np.stack([resnet_values[:i].mean().item() for i in range(1, n)])
        resnet_values_conv = np.convolve(resnet_values, np.ones(n)/n, mode="valid")
        resnet_values_mean = np.concatenate([resnet_values_ext, resnet_values_conv])

        plainnet_values = np.array(plainnet_ds["Value"])
        plainnet_values_ext = np.stack([plainnet_values[:i].mean().item() for i in range(1, n)])
        plainnet_values_conv = np.convolve(plainnet_values, np.ones(n)/n, mode="valid")
        plainnet_values_mean = np.concatenate([plainnet_values_ext, plainnet_values_conv])

        plt.subplot(3, 2, i+1)
        plt.subplots_adjust(bottom=0.1)
        
        if tag[-5:] == "train":
            plt.xticks(range(0, 600, 100))
            plt.xlabel("Step")
            plt.vlines([360, 360], -1, 5, color="grey", linewidth=0.5, linestyle="dashed")
        else:
            plt.xticks(range(0, 25, 5))
            plt.xlabel("Epoch")

        plt.plot(steps, resnet_values, linewidth=0.5, linestyle="dashed", label="ResNet", c="red")
        plt.plot(steps, resnet_values_mean, linewidth=1.5, label="ResNet (smoothed)", c="red")

        plt.plot(steps, plainnet_values, linewidth=0.5, linestyle="dashed", label="PlainNet", c="blue")
        plt.plot(steps, plainnet_values_mean, linewidth=1.5, label="PlainNet (smoothed)", c="blue")

        if "Loss" in tag:
            plt.yticks(np.arange(0, 5, 0.5))
            plt.ylim([-0.1, 4.5])
        else:
            plt.yticks(np.arange(0, 1.2, 0.2))
            plt.ylim([-0.1, 1.1])
        plt.ylabel(LABELS[i])

        plt.legend()
    
    plt.show()